In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

data_path \
    = 'https://raw.githubusercontent.com/fclesio/learning-space/master/Datasets/02%20-%20Classification/default_credit_card.csv'

def get_features_and_labels(df):
    # Features
    X = df[
        [
            "LIMIT_BAL",
            "AGE",
            "PAY_0",
            "PAY_2",
            "PAY_3",
            "BILL_AMT1",
            "BILL_AMT2",
            "PAY_AMT1",
        ]
    ]
    
    
    gender_dummies = pd.get_dummies(df[["SEX"]].astype(str))
    X = pd.concat([X, gender_dummies], axis=1)

    # Labels
    y = df["DEFAULT"]
    return X, y


def get_results(y_test, y_pred):
    acc = metrics.accuracy_score(y_test, y_pred)
    acc = round(acc, 2) * 100
    
    df_results = pd.DataFrame(y_pred)
    df_results.columns = ["status"]
    
    print(f"Accuracy: {acc}%")
    print(df_results.groupby(by=["status"]).size())

df = pd.read_csv(data_path)

X, y = get_features_and_labels(df)

X_train, X_test, y_train, y_test \
    = train_test_split(X, y, test_size=0.1, random_state=42)


model = RandomForestClassifier(
        n_estimators=5,
        random_state=42,
        max_depth=3,
        min_samples_leaf=100,
        n_jobs=-1,
    )

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

get_results(y_test, y_pred)

pickle.dump(model, open("model_rf.pkl", 'wb'))

Accuracy: 82.0%
status
0    2655
1     345
dtype: int64


### Attack 

In that case we have a pre-trained model that will be handover to another place, for example, from some Data Science team to a Machine Learning engineering team.

The attack consist to take this model, make a slighty modification that can be harmful, and put it again in the ML Supply Chain flow, that in this case is the ML Production Pipeline.

In [2]:
# Load model from Pickle
model_rf_reload_pkl = pickle.load(open('model_rf.pkl', 'rb'))

# Displays prediction classes
model_rf_reload_pkl.classes_

array([0, 1])

In [3]:
# Attack: Change the classes for the model only to 1
model_rf_reload_pkl.classes_ = np.array([1, 1])

In [4]:
# Quick check
model_rf_reload_pkl.classes_

array([1, 1])

In [5]:
# Call predict from the new model
y_pred = model_rf_reload_pkl.predict(X_test)

In [6]:
# Check results with a new model
get_results(y_test, y_pred)

Accuracy: 22.0%
status
1    3000
dtype: int64


As we can see, if we have in our ML Supply Chain some "man in the middle" that can take our file and modify it, an entire class of a model (in this case we used Scikit-Learn) can be corrupted. This was an extreme case but remember: The attacker wants to stay attacking for a long time and wants to stay in the stealth model at maximum as they can.


### Countermeasures
   - If there's some risk of some "man in the middle" in the touchpoints of ML models (e.g. a DS team, makes the handover to an MLE team and after to another team), it's suitable to use SHA1 or MD5 references from the start to assure the file integrity between all entities involved with the model;  
   
   - If possible, own your models and deployment and reduce as many intermediate steps as possible;  
   
   - If possible, avoid implementations that allow modifications in models (e.g. classes, attributes, coefficients, etc)
    